<a href="https://colab.research.google.com/github/fkonrad97/Network/blob/main/routingtable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import csv
import pandas as pd
import numpy as np
import networkx as nx
import random
from scipy.stats import entropy

In [32]:
# Read positions of nodes (X,Y,Z)

def positionRead(name):
    positions = pd.read_csv(name + ".csv", header=None, sep=";")
    # Remove a plus sign from the end of the number
    positions[0][0] = positions[0][0][:-1]
    positions[0] = positions[0].astype(float)    # Convert data to numerical value
    return positions

In [33]:
# Read connection table between nodes

def connectionRead(name):
    connections = pd.read_csv(name + ".csv", header=None)
    return connections

In [34]:
# List of the positions of nodes
# positions = positionRead("Network/Brain_data/Brain1Positions")
positions = positionRead("/content/Brain1Positions")

# List of how nodes connected to each other
# connections = connectionRead("Network/Brain_data/Brain1Connections")
connections = connectionRead("/content/Brain1Connections")

neighbours = []

for i in range(len(connections)):
  for j in range(len(connections)):
     if connections[i][j] == 1 and i < j:
       neighbours.append([i, j])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
def findPaths(src, graph):
  paths = []

  for j in range(len(graph.nodes)):
    if src!=j:
      path = [p for p in nx.all_shortest_paths(graph, source=src, target=j)]
      paths.append(path)
  
  return paths

In [36]:
def chooseRandomShortest(paths):
  randomPaths = []

  for i in paths:
    randomPaths.append(random.choice(i))

  return randomPaths

In [37]:
def avgEntropy(graph, routingTables):
  sum = 0

  for i in range(len(graph.nodes)):
    routingTables[i].getNextHopsList()
    sum += entropy(routingTables[i].nextHops)
  
  return (sum / len(graph.nodes))

In [38]:
# Init Graph

G = nx.Graph()
for i in neighbours:
  G.add_edge(i[0],i[1])

# number of nodes
size = len(G.nodes)

# List of paths from nodes to nodes
paths = []
for i in range(size):
  temp = []
  for j in range(size):
    if i!=j:
      path = [p for p in nx.all_shortest_paths(G, source=i, target=j)]
      temp.append(path)
  paths.append(temp)

In [39]:
class routingTable():

  def __init__(self, nodeID):
    self.nodeID = nodeID
    self.table = []
    self.nextHops = []

  def addtoTable(self, dest, nextHop):
    self.table.append([dest, nextHop])

  def findNextHop(self, graph):
    paths = findPaths(self.nodeID, graph)
    paths = chooseRandomShortest(paths)
    for i in paths:
      self.addtoTable(i[-1], i[1])

  def getNextHopsList(self):
    self.nextHops = []
    for i in self.table:
      self.nextHops.append(i[1])

In [48]:
# Each nodes' routing table
routingTables = []

for j in range(100):
  routingtable = []
  for i in range(len(G.nodes)):
    table = routingTable(i)
    table.findNextHop(G)
    routingtable.append(table)
  routingTables.append(routingtable)

In [49]:
entropyList = []

for i in routingTables:
  entropyList.append(avgEntropy(G, i))